In [ ]:
%pip install idx2numpy


In [27]:
import idx2numpy
import numpy as np
import nn
import matplotlib.pyplot as plt

In [30]:
train_img = (idx2numpy.convert_from_file("train-images.idx3-ubyte")/255).reshape(-1, 28 * 28)
train_lbl = idx2numpy.convert_from_file("train-labels.idx1-ubyte")
test_img = (idx2numpy.convert_from_file("t10k-images.idx3-ubyte")/255).reshape(-1, 28 * 28)
test_lbl = idx2numpy.convert_from_file("t10k-labels.idx1-ubyte")




In [31]:
m = nn.Model()


m.add(nn.Layer_Dense(28*28, 500, weight_regularizer_l2 = 0.02, bias_regularizer_l2 = 0.02))
m.add(nn.Activation_ReLU())
m.add(nn.Layer_Dense(500, 500))
m.add(nn.Activation_ReLU())
m.add(nn.Layer_Dense(500, 10))
m.add(nn.Activation_Softmax())


m.set(Loss = nn.Loss_CategoricalCrossentropy(),      
 Optimizer = nn.Optimizer_Adam())


for i in range(1, 5):
    print(f'EPOCH = {i} --------------------------------------')
    m.train(train_img, train_lbl , batch= 6, shuffle = True, print_every = 2)
    m.test(test_img, test_lbl)

EPOCH = 1 --------------------------------------
Loss : 3.7004     [10000/ 60000]
Loss : 2.0023     [30000/ 60000]
Loss : 1.3310     [50000/ 60000]


c:\Users\pro\Documents\GitHub\scrapy\nn.py:339: RuntimeWarning: divide by zero encountered in scalar divide
  test_loss /= batch


Test LOSS : inf    Accuracy = 72.0 
EPOCH = 2 --------------------------------------
Loss : 0.8516     [10000/ 60000]
Loss : 0.7291     [30000/ 60000]
Loss : 0.5944     [50000/ 60000]
Test LOSS : inf    Accuracy = 84.7 
EPOCH = 3 --------------------------------------
Loss : 0.4867     [10000/ 60000]
Loss : 0.4824     [30000/ 60000]
Loss : 0.4420     [50000/ 60000]
Test LOSS : inf    Accuracy = 88.9 
EPOCH = 4 --------------------------------------
Loss : 0.3738     [10000/ 60000]
Loss : 0.3813     [30000/ 60000]
Loss : 0.3579     [50000/ 60000]
Test LOSS : inf    Accuracy = 90.7 
